# Movie recommendation on Amazon SageMaker with Factorization Machines

### Step SM1: Download ml-100k data  
***The data sets are needed to train our Factorization Machine. We use the 100,000 movie ratings given by users from MovieLens data sets.***

In [ ]:
!pip install pydot

In [ ]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

### Data Information
*ua.base : data for training*  
*ua.test : data for test/validation*  
*Headers/columns :* ***user id | item id | rating (1-5) | timestamp***

### Step SM2: Let's shuffle rating items data

Columns in ua.base and ua.test file:
user id | item id | rating | timestamp

***The code below will show how ua.test file look like for first 10 lines:***

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv('./ml-100k/ua.base', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
test_df = pd.read_csv('./ml-100k/ua.test', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
train_df

In [ ]:
pd.read_csv('./ml-100k/u.user', sep='|', names=['user_id','age','gender','occupation','zip'])

In [ ]:
pd.read_csv('./ml-100k/u.item', sep='|', names=['item_id','title','release_date','video_release_date','imdb_url','UNKOWN','Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Noir','Horror','Musical','Mystery','Romance','SciFi','Thriller','War','Western'])

### Step SM3: Build training set and test set

***Import necessary modules***

In [ ]:
import boto3, csv, io, json
import numpy as np
from scipy.sparse import lil_matrix

import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer


***Set S3 bucket and prefix***

In [ ]:
%%time

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'factorization-machine-sagemaker'

***Initialize number of total users and movies in data set, as well as number of train and test data***

In [ ]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies

nbRatingsTrain=90570
nbRatingsTest=9430

***For each user, build a list of rated movies with their ratings. We'd need this to add random negative samples.***  
This is achieved by a dictionary moviesByUser that will look like this:  
```
{
  '0':[[875072546,4],[875072441,3]],
  '1':[[887431882,2]]
}
```
where key represents userId (stored as userId - 1) and each element in the values represents movieId (stored as movieId -1) as first element and the rating as second element

In [ ]:
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)]=[]
 
with open('./ml-100k/ua.base','r') as f:
    samples=csv.reader(f,delimiter='\t')
    for userId,movieId,rating,timestamp in samples:
        moviesByUser[str(int(userId)-1)].append([int(movieId)-1,rating]) 

### Step SM4: Define method to load dataset

***The data will be loaded into 2 vectors: feature vector X and label vector Y***  
Feature vector X is a one-hot encoded vector that sticks and flattens user Ids and movie Ids together. It should look like this below (without rows' and columns' labels):   

|<pre></pre>| 1 	    |      2    |    3  	|<pre>...</pre>| 1  | 2  |<pre>...</pre>|
|   :---:   |:---:      |    :---:	|   :---:	|  :---:	|  :---:	 |  :---:     |   :---:	  |
| **data0** | 1 	    |<pre></pre>|<pre></pre>|<pre></pre>|<pre></pre> |<pre>1</pre>|<pre></pre>|
| **data1** |<pre></pre>| 1 	    |<pre></pre>|<pre></pre>|<pre>1</pre>|<pre></pre> |<pre></pre>|
|<pre>...</pre>|<pre></pre>|<pre></pre>|<pre></pre>|<pre></pre>|<pre></pre> |<pre></pre> |<pre></pre>|
 
It is a 2D sparse matrix where columns are user Ids and movie Ids, and rows are data items in the training/test data set.
One row represents 1 training/test data that has 2 ones (1s) that mark the user Id and movie Id that he/she rated.   

Label vector Y is a 1D vector containing expected output. It looks like this below (without rows' labels):

|<pre></pre>|<pre></pre>|
| :--- | :---:|
|**data0**| 1 |
|**data1**| 1 |
|**data2**| 0 |
|**data3**| 1 |
|<pre>...</pre>|<pre></pre>|


If user's rating for that movie is 4 or 5, then value is 1, otherwise 0. Each element corresponds to one data.


In [ ]:
def loadDataset(filename, lines, columns):

    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    
    return X,Y

***Now that we have defined the loadDataset method, lets load both training and test data***

In [ ]:
X_train, Y_train = loadDataset('./ml-100k/ua.base', nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('./ml-100k/ua.test',nbRatingsTest,nbFeatures)

***Let's examine the dimensions of X and Y vectors***

In [ ]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

### Step SM5: Convert to protobuf and save to S3

In [ ]:
train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train3')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test3')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

### Step SM6: Run training job
***We are done with the data preparation part. Let's begin training our Factorization Machine model.***  
***SageMaker provides both the container and built-in algorithm to run the training and inference.***

Below is the list of container images containing built-in algorithm for factorization machine in SageMaker per region:

In [ ]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest',
              'ap-southeast-1': '475088953585.dkr.ecr.ap-southeast-1.amazonaws.com/factorization-machines:latest'}

***Behing the scene, SageMaker provisions a container to run the training, and terminate it after training job succeeds. Metrics during training, including accuracy are posted to CloudWatch Metrics.***    

Note: If you like GUI (Graphical User Interface), you can execute the training via AWS Console too. Basically we can interact with AWS in 3 ways: AWS Console (GUI), CLI, and SDK. For this lab, we are using SDK. You can inspect https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs (change the region as necessary) to see the running training job after you run the step below

In [ ]:
fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=10)

fm.fit({'train': train_data, 'test': test_data})

***If the training was successful, you wil see 'Training job completed' at the end of the output. Scroll up to see the  train and test accuracy***    

***After training phase completed, we have the model parameters stored in S3 (in the output path you specified). You can check your S3 bucket that contains the output to inspect how the training job output looks like***

### Step SM7: Deploy model

***Now, let's deploy the model for inference using SageMaker SDK. It will spin-up a new virtual machine with container containing algorithm for inference. It will give us an API endpoint for inference.***

In [ ]:
fm_predictor = fm.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

### Step SM8: Run predictions

***After the model is deployed and given an endpoint, we can run the prediction / inference.***  
Below we define the serializer and deserializer for the prediction request/response data

In [ ]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print js
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

***Let's test the prediction with some data from the test set***

In [ ]:
index_from = 900
index_to = 910
result = fm_predictor.predict(X_test[index_from:index_to].toarray())

***Display the prediction in pretty table, being compared againts the actual rating (label) from the test set.***.      
Observe that for score between 0.3 to 0.7 our recommender may guess incorrectly.

In [ ]:
!pip install tabulate
import tabulate
from IPython.display import HTML, display

scores, predicted_rating = ['Score'], ['Predicted Rating']
for r in result['predictions']:
    scores.append("%.2f" % r['score'])
    predicted_rating.append(r['predicted_label'])


table = [scores, predicted_rating, ['Actual Rating'] + Y_test[index_from:index_to].tolist() ]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

### Step SM9: Get Movies Recommendation

***After testing the prediction, let's get real movies recommendation for a particular user***    
First, let's prepare a dictionary that maps movie ID to its title. We use the u.item data containing movies' details

In [ ]:
movies = {}
with open('./ml-100k/u.item','r') as f:
    samples=csv.reader(f,delimiter='|')
    for movieId,m_title,r_date,video_r_date,imdb_URL,unkwn,act,adv,anm,kid,cmd,crime,doc,drama,fantasy,f_noir,horror,msc,myst,rom,sfy,thriller,war,west in samples:
        movies[int(movieId)] = m_title

***Define some parameters***    
userId = the ID of user who needs the recommendations    
score_threshold = Cut-off score. Value nearer to 1 means that we only consider strong predictions. Value 0.5 is the minimun.    
maximum_recommendations = Maximum of movies recommendation. The actual result may be less than this if not many movies are strongly recommended.

In [ ]:
userId = 344
score_threshold = 0.50
maximum_recommendations = 20

***Run predictions for all movies for this particular user and sort the output based on score***

In [ ]:
recommended_movies=[]
for movieId in range(nbMovies):
    test_input = lil_matrix((1, nbFeatures)).astype('float32')
    test_input[0, int(userId)-1] = 1
    test_input[0, nbUsers+int(movieId)-1] = 1
    result = fm_predictor.predict(test_input.toarray())
    result_label, result_score = int(result['predictions'][0]['predicted_label']), float(result['predictions'][0]['score'])
    if (result_label == 1) and (result_score > score_threshold):
        recommended_movies.append([int(movieId),result_score])
        
def getVal(item):
    return item[1]
recommended_movies = sorted(recommended_movies,key=getVal,reverse=True)



***Print out the result of top recommended movies***

In [ ]:
output_table = [['<strong>Movie Title</strong>','<strong>Score</strong>']]
for i in range(min(maximum_recommendations,len(recommended_movies))):
    output_table.append([movies[int(recommended_movies[i][0])],recommended_movies[i][1]])

display(HTML(tabulate.tabulate(output_table, tablefmt='html')))

***Compare the recommendation with the top 20 movies that are actually rated by that particular user, sorted from the highest rating***

In [ ]:
def find_top_rated_movies(user_id, k):
    rated_movies = moviesByUser[str(int(user_id)-1)]
    rated_movies = sorted(rated_movies,key=getVal,reverse=True)
    results = []
    
    for movie in rated_movies:
        results.append([movies[int(movie[0]+1)],movie[1]])
    return results[0:k]

output_table = [['<strong>Movie Title</strong>','<strong>Actual Rating</strong>']]
for m in find_top_rated_movies(userId,20):
    output_table.append(m)

display(HTML(tabulate.tabulate(output_table, tablefmt='html')))
